In [2]:
import pandas as pd

In [25]:
train_data=pd.read_csv('./train.csv',encoding='ISO-8859-1')
train_data#99989 rows × 3 columns

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...


In [26]:
train_data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer, punkt, TweetTokenizer
from nltk.corpus import stopwords,state_union
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import sys
import string
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import preprocessor as p

train_data=pd.read_csv('./train00.csv',encoding='ISO-8859-1')
#Emoticons
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
#print(sorted(emos_count,reverse=True))
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)

def stop_words():
    mw = most_used_words(train_data.SentimentText.str.cat())
    most_words = []
    for w in mw:
        if len(most_words) == 1000:
            break
        if w in stopwords.words("english"):
            continue
        else:
            most_words.append(w)
    print(sorted(most_words))

def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]


class TextPreProc(BaseEstimator, TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", "")

        # Keeping only the word after the #
        X = X.str.replace("#", "")
        X = X.str.replace(r"[-\.\n]", "")
        # Removing HTML garbage
        X = X.str.replace(r"&\w+;", "")
        # Removing links
        X = X.str.replace(r"https?://\S*", "")
        # replace repeated letters with only two occurences
        # heeeelllloooo => heelloo
        X = X.str.replace(r"(.)\1+", r"\1\1")
        # mark emoticons as happy or sad
        X = X.str.replace(HAPPY_EMO, " happyemoticons ")
        X = X.str.replace(SAD_EMO, " sademoticons ")
        X = X.str.lower()
        return X

sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

# I get those parameters from the 'Fine tune the model' part
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPreProc(use_mention=True)),
    ('vectorizer', vectorizer),
])
#  split data into learning set and testing set
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = pipeline.fit_transform(learn_data)


#lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    #'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}
for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    print("mean = ", scores.mean())
    print("variance = ", scores.var())
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")


grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPreProc()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)


#march05data=pd.read_csv('./20200305 Coronavirus Tweets.CSV',encoding='latin-1',low_memory=False)
mnb.fit(learning_data, sentiments_learning)
testing_data = pipeline.transform(test_data)
mnb.score(testing_data, sentiments_test)
sub_data = pd.read_csv(r'./20200305 Coronavirus Tweets.CSV', encoding='ISO-8859-1',low_memory=False)
sub_learning = pipeline.transform(sub_data['SentimentText'].apply(lambda sub_learning: np.str_(sub_learning)))

sub = pd.DataFrame(sub_data['ItemID'], columns=("ItemID", "Sentiment"))
sub["Sentiment"] = mnb.predict(sub_learning)
print(sub)

model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])


#march05data[7680:27822]


Happy emoticons: {':D', 'xD', ';)', ';D', ':-D', ':p', ';-D', 'xd', 'x)', 'XD', ':d', ';P', ';p', ';-)', ';d'}
Sad emoticons: {':|', ":'(", ':(', ':/'}
=== bernoulliNB ===
scores =  [0.78385629 0.7854871  0.78986867 0.78602415 0.78314804 0.79222755
 0.78778662 0.78676299 0.78410286 0.79353933]
mean =  0.7872803608014192
variance =  1.1456331347366269e-05
score on the learning data (accuracy) =  0.9024317064807407

=== multinomialNB ===
scores =  [0.80515441 0.81273032 0.80979955 0.79964401 0.80563255 0.80843468
 0.80421286 0.80781181 0.80580481 0.80908888]
mean =  0.8068313880215445
variance =  1.1605385635631904e-05
score on the learning data (accuracy) =  0.8984455366327581

{'text_pre_processing__use_mention': True, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 2)}
        ItemID  Sentiment
0            1          1
1            2          1
2            3          1
3            4          1
4            5          1
...        ...        ...
528303  526787      

NameError: name 'mnb' is not defined

In [10]:
import pandas as pd
data0328=pd.read_csv('./20200328 Coronavirus Tweets.CSV')
data0328.head()

,1,"Nearly 100,000 coronavirus cases in US: tracker #COVID19 https://t.co/sKwut3Waem"
0,2,'Like wartime': Filipino doctors overwhelmed b...
1,3,Catarman isinailalim sa enhanced community qua...
2,4,#Toquedequeda A partir de las 22:00 horas y ha...
3,5,"La mortalidad por el #COVID19 es baja, es impo..."
4,6,Correspondence: Epidemiology of Covid-19. #COV...


In [26]:
data0328=pd.read_csv('./20200328 Coronavirus Tweets.CSV')
data0328.columns = ['ItemID', 'Sentiment']
data0328


,ItemID,Sentiment
0,2,'Like wartime': Filipino doctors overwhelmed b...
1,3,Catarman isinailalim sa enhanced community qua...
2,4,#Toquedequeda A partir de las 22:00 horas y ha...
3,5,"La mortalidad por el #COVID19 es baja, es impo..."
4,6,Correspondence: Epidemiology of Covid-19. #COV...
...,...,...
582049,582051,See here 👇🏾specialist nurses from Cuba arrive ...
582050,582052,All day of #selfisolating on the couch binging...
582051,582053,It’s about that time of the day that I start s...
582052,582054,"Smoked Salmon,Carrots,Corn bread,Rice n beans,..."


In [24]:
nrow, ncol = data0328.shape
nrow, ncol
#array = data0328.values
#X = array[:200, :]
#y = array[:, 1]
#X,y


(582054, 2)

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer, punkt, TweetTokenizer
from nltk.corpus import stopwords,state_union
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import sys
import string
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import preprocessor as p

train_data=pd.read_csv('./train00.csv',encoding='ISO-8859-1')
#Emoticons
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
#print(sorted(emos_count,reverse=True))
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)

def stop_words():
    mw = most_used_words(train_data.SentimentText.str.cat())
    most_words = []
    for w in mw:
        if len(most_words) == 1000:
            break
        if w in stopwords.words("english"):
            continue
        else:
            most_words.append(w)
    print(sorted(most_words))

def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]


class TextPreProc(BaseEstimator, TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", "")

        # Keeping only the word after the #
        X = X.str.replace("#", "")
        X = X.str.replace(r"[-\.\n]", "")
        # Removing HTML garbage
        X = X.str.replace(r"&\w+;", "")
        # Removing links
        X = X.str.replace(r"https?://\S*", "")
        # replace repeated letters with only two occurences
        # heeeelllloooo => heelloo
        X = X.str.replace(r"(.)\1+", r"\1\1")
        # mark emoticons as happy or sad
        X = X.str.replace(HAPPY_EMO, " happyemoticons ")
        X = X.str.replace(SAD_EMO, " sademoticons ")
        X = X.str.lower()
        return X

sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

# I get those parameters from the 'Fine tune the model' part
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPreProc(use_mention=True)),
    ('vectorizer', vectorizer),
])
#  split data into learning set and testing set
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = pipeline.fit_transform(learn_data)


#lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    #'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}
for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    print("mean = ", scores.mean())
    print("variance = ", scores.var())
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")


grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPreProc()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)


#march05data=pd.read_csv('./20200305 Coronavirus Tweets.CSV',encoding='latin-1',low_memory=False)
mnb.fit(learning_data, sentiments_learning)
testing_data = pipeline.transform(test_data)
mnb.score(testing_data, sentiments_test)
sub_data = pd.read_csv(r'./20200328 Coronavirus Tweets.CSV', encoding='ISO-8859-1',low_memory=False)
sub_learning = pipeline.transform(sub_data['SentimentText'].apply(lambda sub_learning: np.str_(sub_learning)))

sub = pd.DataFrame(sub_data['ItemID'], columns=("ItemID", "Sentiment"))
sub["Sentiment"] = mnb.predict(sub_learning)
print(sub)

model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])


Happy emoticons: {':-D', ':d', ':D', ';-)', ';-D', 'xd', ';P', ':p', 'x)', 'XD', ';D', ';d', 'xD', ';p', ';)'}
Sad emoticons: {":'(", ':(', ':|', ':/'}


c:\users\wyvva\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\wyvva\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

=== logitic regression ===
scores =  [0.80922482 0.81095492 0.81387171 0.8112411  0.81428744 0.80677556
 0.81034483 0.81186441 0.81299831 0.81221337]
mean =  0.8113776473512108
variance =  4.532706454260302e-06


c:\users\wyvva\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


score on the learning data (accuracy) =  0.8709138187221397

=== bernoulliNB ===
scores =  [0.7944438  0.78326281 0.78852035 0.79192583 0.78753938 0.78446057
 0.79177921 0.78370491 0.79009738 0.79294533]
mean =  0.7888679566391448
variance =  1.4643812798786736e-05
score on the learning data (accuracy) =  0.9027603154646245

=== multinomialNB ===
scores =  [0.80806057 0.80882024 0.80449039 0.80830742 0.81079875 0.80387787
 0.81207071 0.81027359 0.81014072 0.81064758]
mean =  0.8087487828171553
variance =  6.563883048949389e-06
score on the learning data (accuracy) =  0.8997456852211682

{'text_pre_processing__use_mention': True, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 2)}


KeyError: 'SentimentText'

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer, punkt, TweetTokenizer
from nltk.corpus import stopwords,state_union
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import sys
import string
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import preprocessor as p

getdata=pd.read_csv('./Twitter_Data.csv',encoding='ISO-8859-1')
train_data= getdata.dropna()
pd.DataFrame({"tweet": SentimentText, "category": Sentiment})
#Emoticons
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
#print(sorted(emos_count,reverse=True))
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)

def stop_words():
    mw = most_used_words(train_data.SentimentText.str.cat())
    most_words = []
    for w in mw:
        if len(most_words) == 1000:
            break
        if w in stopwords.words("english"):
            continue
        else:
            most_words.append(w)
    print(sorted(most_words))

def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]


class TextPreProc(BaseEstimator, TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", "")

        # Keeping only the word after the #
        X = X.str.replace("#", "")
        X = X.str.replace(r"[-\.\n]", "")
        # Removing HTML garbage
        X = X.str.replace(r"&\w+;", "")
        # Removing links
        X = X.str.replace(r"https?://\S*", "")
        # replace repeated letters with only two occurences
        # heeeelllloooo => heelloo
        X = X.str.replace(r"(.)\1+", r"\1\1")
        # mark emoticons as happy or sad
        X = X.str.replace(HAPPY_EMO, " happyemoticons ")
        X = X.str.replace(SAD_EMO, " sademoticons ")
        X = X.str.lower()
        return X

sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

# I get those parameters from the 'Fine tune the model' part
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPreProc(use_mention=True)),
    ('vectorizer', vectorizer),
])
#  split data into learning set and testing set
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = pipeline.fit_transform(train_data['SentimentText'].apply(lambda learning_data: np.str_(learning_data)))


#lr = LogisticRegression()
#bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    #'logitic regression': lr,
    #'bernoulliNB': bnb,
    'multinomialNB': mnb,
}
for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    print("mean = ", scores.mean())
    print("variance = ", scores.var())
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")


grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPreProc()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)


#march05data=pd.read_csv('./20200305 Coronavirus Tweets.CSV',encoding='latin-1',low_memory=False)
mnb.fit(learning_data, sentiments_learning)
testing_data = pipeline.transform(test_data)
mnb.score(testing_data, sentiments_test)

sub_learning = pipeline.transform(sub_data.SentimentText.apply(lambda sub_learning: np.str_(sub_learning)))
data0305=pd.read_csv(r'./20200305 Coronavirus Tweets.CSV', encoding='ISO-8859-1',low_memory=False)
newdata=data0305[7680:27822]

sub_data = newdata
sub = pd.DataFrame(sub_data['ItemID'], columns=("ItemID", "Sentiment"))
sub["Sentiment"] = mnb.predict(sub_learning)
print(sub)

model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])